### install dependencies

In [1]:
!gdown "1W3-WEplVSztLR3lvkyYdiKZGMT4y0cNi&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1W3-WEplVSztLR3lvkyYdiKZGMT4y0cNi&confirm=t
To: /content/IMDB.zip
100% 61.4M/61.4M [00:00<00:00, 224MB/s]


In [2]:
!unzip IMDB.zip

In [3]:
!pip install mlflow

# Content-based filtering

### import libraries

In [4]:
import numpy as np
import pandas as pd
import mlflow as mf

In [5]:
#mf.log_artifacts({'rating':'/content/rating_small.csv', 'rating':'/content/rating_small.csv', 'movies':'/content/movies_metadata.csv','keywords':'/content/keywords.csv', 'credits':'/content/credits.csv'})

### read data from file

In [6]:
keywords = pd.read_csv('/content/IMDB/keywords.csv')
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [7]:
rating = pd.read_csv('/content/IMDB/ratings_small.csv')
rating

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [8]:
credits = pd.read_csv('/content/IMDB/credits.csv')
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [9]:
metadata = pd.read_csv('/content/IMDB/movies_metadata.csv')
metadata

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


keep only related columns from released movies:

In [10]:
metadata = metadata[metadata['status'] == 'Released']
cols = np.array(['adult', 'belongs_to_collection', 'genres', 'id', 'original_language', 'title', 'production_countries', 'production_companies', 'video']) 
metadata = metadata[cols]

In [11]:
metadata.iloc[1]

adult                                                                False
belongs_to_collection                                                  NaN
genres                   [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
id                                                                    8844
original_language                                                       en
title                                                              Jumanji
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
production_companies     [{'name': 'TriStar Pictures', 'id': 559}, {'na...
video                                                                False
Name: 1, dtype: object

In [12]:
def find_collection(x):
    if x == '':
        return ''
    return eval(str(x))['name']

metadata['belongs_to_collection'] = metadata['belongs_to_collection'].fillna('')
metadata['belongs_to_collection'] = metadata['belongs_to_collection'].apply(find_collection)
metadata.iloc[1]

adult                                                                False
belongs_to_collection                                                     
genres                   [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
id                                                                    8844
original_language                                                       en
title                                                              Jumanji
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
production_companies     [{'name': 'TriStar Pictures', 'id': 559}, {'na...
video                                                                False
Name: 1, dtype: object

In [13]:
def find_names(x):
    if x == '':
        return ''
    genre_arr = eval(str(x))
    return ','.join(i['name'] for i in eval(str(x)))
    
metadata['genres'] = metadata['genres'].fillna('')
metadata['genres']=metadata['genres'].apply(find_names)
metadata['production_countries']=metadata['production_countries'].apply(find_names)
metadata['production_companies']=metadata['production_companies'].apply(find_names)
credits['cast'] = credits['cast'].apply(find_names)
metadata.iloc[1]

adult                                                                False
belongs_to_collection                                                     
genres                                            Adventure,Fantasy,Family
id                                                                    8844
original_language                                                       en
title                                                              Jumanji
production_countries                              United States of America
production_companies     TriStar Pictures,Teitler Film,Interscope Commu...
video                                                                False
Name: 1, dtype: object

In [14]:
keywords['keywords'] = keywords['keywords'].apply(find_names)
metadata['id'] = metadata['id'].astype(int)
metadata = pd.merge(metadata,keywords,how='inner',on='id')
metadata.iloc[1]

adult                                                                False
belongs_to_collection                                                     
genres                                            Adventure,Fantasy,Family
id                                                                    8844
original_language                                                       en
title                                                              Jumanji
production_countries                              United States of America
production_companies     TriStar Pictures,Teitler Film,Interscope Commu...
video                                                                False
keywords                 board game,disappearance,based on children's b...
Name: 1, dtype: object

In [15]:
def to_int(x):
    if x == 'True':
        return 1
    return 0

In [16]:
metadata['adult'].unique()

array(['False', 'True'], dtype=object)

there are 3 values other than True or False in adult column. there are entered by mistake so we remove those rows.

In [17]:
metadata = metadata[(metadata['adult'] == 'True') | (metadata['adult'] == 'False')]
metadata['adult'] = metadata['adult'].apply(to_int)
metadata['video'].unique()

array([False, True], dtype=object)

removing nan values from dataset and replacing 'True' and 'False' with 1 and 0:

In [18]:
metadata = metadata[~metadata['video'].isna()]
metadata['video'] = metadata['video'].apply(to_int)

## Vectorize string features

In [19]:
metadata

,adult,belongs_to_collection,genres,id,original_language,title,production_countries,production_companies,video,keywords
0,0,Toy Story Collection,"Animation,Comedy,Family",862,en,Toy Story,United States of America,Pixar Animation Studios,0,"jealousy,toy,boy,friendship,friends,rivalry,bo..."
1,0,,"Adventure,Fantasy,Family",8844,en,Jumanji,United States of America,"TriStar Pictures,Teitler Film,Interscope Commu...",0,"board game,disappearance,based on children's b..."
2,0,Grumpy Old Men Collection,"Romance,Comedy",15602,en,Grumpier Old Men,United States of America,"Warner Bros.,Lancaster Gate",0,"fishing,best friend,duringcreditsstinger,old men"
3,0,,"Comedy,Drama,Romance",31357,en,Waiting to Exhale,United States of America,Twentieth Century Fox Film Corporation,0,"based on novel,interracial relationship,single..."
4,0,Father of the Bride Collection,Comedy,11862,en,Father of the Bride Part II,United States of America,"Sandollar Productions,Touchstone Pictures",0,"baby,midlife crisis,confidence,aging,daughter,..."
...,...,...,...,...,...,...,...,...,...,...
46017,0,,"Drama,Family",439050,fa,Subdue,Iran,,0,tragic love
46018,0,,Drama,111109,tl,Century of Birthing,Philippines,Sine Olivia,0,"artist,play,pinoy"
46019,0,,"Action,Drama,Thriller",67758,en,Betrayal,United States of America,American World Pictures,0,
46020,0,,,227506,en,Satan Triumphant,Russia,Yermoliev,0,


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def my_tok(text):
    return text.split(",")

def vectorize_string(col_name, feature_name, limit=None, df=metadata):
    vectorizer = CountVectorizer(tokenizer=my_tok, max_features=limit, min_df=2)
    X = vectorizer.fit_transform(df[col_name])
    vec_cols = vectorizer.get_feature_names_out()
    vec_data = X.toarray()
    #vec_cols = np.char.add(feature_name+':', vec_cols)
    vec_cols = feature_name+':'+vec_cols
    return vec_data, vec_cols

def tfidf(col_name, feature_name, limit=None, df=metadata):
    vectorizer = TfidfVectorizer(tokenizer=my_tok, max_features=limit, min_df=2)
    X = vectorizer.fit_transform(df[col_name])
    vec_cols = vectorizer.get_feature_names_out()
    vec_data = X.toarray()
    #vec_cols = np.char.add(feature_name+':', vec_cols)
    vec_cols = feature_name+':'+vec_cols
    return vec_data, vec_cols

In [21]:
genre_data, genre_cols = vectorize_string('genres', 'genre')
genre_cols

array(['genre:', 'genre:action', 'genre:adventure', 'genre:animation',
       'genre:comedy', 'genre:crime', 'genre:documentary', 'genre:drama',
       'genre:family', 'genre:fantasy', 'genre:foreign', 'genre:history',
       'genre:horror', 'genre:music', 'genre:mystery', 'genre:romance',
       'genre:science fiction', 'genre:thriller', 'genre:tv movie',
       'genre:war', 'genre:western'], dtype=object)

In [22]:
companies_data, companies_cols = vectorize_string('production_companies', 'company', 100)
companies_cols

array(['company:', 'company: the', 'company:amblin entertainment',
       'company:american international pictures (aip)', 'company:arte',
       'company:arte france cinéma', 'company:bbc', 'company:bbc films',
       'company:blumhouse productions',
       'company:british broadcasting corporation (bbc)', 'company:canal+',
       'company:canal+ españa', 'company:castle rock entertainment',
       'company:centre national de la cinématographie (cnc)',
       'company:channel four films', 'company:ciné+',
       'company:cinécinéma', 'company:cj entertainment',
       'company:columbia pictures',
       'company:columbia pictures corporation', 'company:dc comics',
       'company:dimension films', 'company:dreamworks skg',
       'company:dune entertainment', 'company:eurimages',
       'company:europacorp', 'company:film i väst', 'company:film4',
       'company:first national pictures', 'company:focus features',
       'company:fox 2000 pictures', 'company:fox film corporation',
   

In [23]:
countries_data, countries_cols = vectorize_string('production_countries', 'country')
countries_cols

array(['country:', 'country:afghanistan', 'country:albania',
       'country:algeria', 'country:angola', 'country:antarctica',
       'country:argentina', 'country:armenia', 'country:aruba',
       'country:australia', 'country:austria', 'country:azerbaijan',
       'country:bahamas', 'country:bangladesh', 'country:belarus',
       'country:belgium', 'country:bhutan', 'country:bolivia',
       'country:bosnia and herzegovina', 'country:botswana',
       'country:brazil', 'country:bulgaria', 'country:burkina faso',
       'country:cambodia', 'country:cameroon', 'country:canada',
       'country:chad', 'country:chile', 'country:china',
       'country:colombia', 'country:congo', 'country:costa rica',
       "country:cote d'ivoire", 'country:croatia', 'country:cuba',
       'country:cyprus', 'country:czech republic',
       'country:czechoslovakia', 'country:denmark',
       'country:dominican republic', 'country:east germany',
       'country:ecuador', 'country:egypt', 'country:el salvad

In [24]:
collection_data, collection_cols = vectorize_string('belongs_to_collection', 'collection')
collection_cols

array(['collection:', 'collection: amore e… - collezione',
       'collection: band of assassins collection', ...,
       'collection:что творят мужчины! (коллекция)',
       'collection:男はつらいよ シリーズ', 'collection:식객 시리즈'], dtype=object)

In [25]:
metadata['original_language']= metadata['original_language'].fillna('')
lang_data, lang_cols = vectorize_string('original_language', 'lang')
lang_cols

array(['lang:', 'lang:ab', 'lang:af', 'lang:am', 'lang:ar', 'lang:bg',
       'lang:bm', 'lang:bn', 'lang:bo', 'lang:bs', 'lang:ca', 'lang:cn',
       'lang:cs', 'lang:da', 'lang:de', 'lang:el', 'lang:en', 'lang:es',
       'lang:et', 'lang:eu', 'lang:fa', 'lang:fi', 'lang:fr', 'lang:he',
       'lang:hi', 'lang:hr', 'lang:hu', 'lang:id', 'lang:is', 'lang:it',
       'lang:iu', 'lang:ja', 'lang:ka', 'lang:kk', 'lang:kn', 'lang:ko',
       'lang:ku', 'lang:ky', 'lang:lo', 'lang:lt', 'lang:lv', 'lang:mk',
       'lang:ml', 'lang:mn', 'lang:mr', 'lang:ms', 'lang:nb', 'lang:ne',
       'lang:nl', 'lang:no', 'lang:pa', 'lang:pl', 'lang:ps', 'lang:pt',
       'lang:ro', 'lang:ru', 'lang:sh', 'lang:sk', 'lang:sl', 'lang:sq',
       'lang:sr', 'lang:sv', 'lang:ta', 'lang:te', 'lang:th', 'lang:tl',
       'lang:tr', 'lang:uk', 'lang:ur', 'lang:vi', 'lang:wo', 'lang:xx',
       'lang:zh'], dtype=object)

In [26]:
collection_cols.shape

(1333,)

In [27]:
keyword_data, keyword_cols = tfidf('keywords', 'keyword', 1000)
keyword_cols

array(['keyword:', 'keyword: new york city', 'keyword:1930s',
       'keyword:1940s', 'keyword:1950s', 'keyword:1960s', 'keyword:1970s',
       'keyword:1980s', 'keyword:19th century', 'keyword:3d',
       'keyword:accident', 'keyword:actor', 'keyword:actress',
       'keyword:addiction', 'keyword:adolescence', 'keyword:adoption',
       'keyword:adult animation', 'keyword:adultery', 'keyword:adventure',
       'keyword:africa', 'keyword:african american',
       'keyword:aftercreditsstinger', 'keyword:afterlife',
       'keyword:age difference', 'keyword:agent', 'keyword:aging',
       'keyword:aids', 'keyword:airplane', 'keyword:airplane crash',
       'keyword:airport', 'keyword:alcohol', 'keyword:alcoholic',
       'keyword:alcoholism', 'keyword:alien', 'keyword:alien invasion',
       'keyword:alien life-form', 'keyword:ambition', 'keyword:ambush',
       'keyword:american', 'keyword:american football', 'keyword:amnesia',
       'keyword:amusement park', 'keyword:anarchic comedy',

In [28]:
credits.drop(columns=['crew'], inplace=True)
credit_data, credit_cols = vectorize_string('cast','cast', 1000, df=credits)
credit_cols

array(['cast:', 'cast: jr.', 'cast:adam sandler', 'cast:adam scott',
       'cast:addison richards', 'cast:adolfo celi', 'cast:adrien brody',
       'cast:agnes moorehead', 'cast:aidan quinn', 'cast:ajay devgn',
       'cast:akim tamiroff', 'cast:akshay kumar', 'cast:al pacino',
       'cast:alain delon', 'cast:alan arkin', 'cast:alan bates',
       'cast:alan cumming', 'cast:alan hale', 'cast:alan mowbray',
       'cast:alan napier', 'cast:alan rickman', 'cast:alan tudyk',
       'cast:alberto sordi', 'cast:alec baldwin', 'cast:alec guinness',
       'cast:alfre woodard', 'cast:alfred molina', 'cast:allen jenkins',
       'cast:allison janney', 'cast:amitabh bachchan', 'cast:amy adams',
       'cast:amy poehler', 'cast:andré dussollier', 'cast:andy garcía',
       'cast:andy lau', 'cast:andy serkis', 'cast:angela lansbury',
       'cast:angelina jolie', 'cast:anjelica huston', 'cast:ann doran',
       'cast:ann-margret', 'cast:anne bancroft', 'cast:anne heche',
       'cast:anthony ho

In [29]:
metadata = pd.concat([metadata[['title','id','adult','video']], 
                      pd.DataFrame(genre_data, columns=genre_cols),
                      pd.DataFrame(countries_data, columns=countries_cols),
                      pd.DataFrame(collection_data, columns=collection_cols),
                      pd.DataFrame(keyword_data, columns=keyword_cols),
                      pd.DataFrame(companies_data, columns=companies_cols),
                      pd.DataFrame(lang_data, columns=lang_cols)], axis=1)

credits[credit_cols] = credit_data
metadata = pd.merge(metadata, credits, how='inner', on='id')
metadata

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


,title,id,adult,video,genre:,genre:action,genre:adventure,genre:animation,genre:comedy,genre:crime,...,cast:william hurt,cast:william powell,cast:william sadler,cast:william schallert,cast:william shatner,cast:winona ryder,cast:woody allen,cast:woody harrelson,cast:xander berkeley,cast:yuen biao
0,Toy Story,862,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,8844,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,15602,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,31357,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,11862,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46159,Subdue,439050,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46160,Century of Birthing,111109,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46161,Betrayal,67758,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46162,Satan Triumphant,227506,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#metadata.drop(['production_countries', 'genres', 'belongs_to_collection', 'keywords', 'production_companies', 'original_language'], axis=1, inplace=True)

list of all numerical features(everything except id and title)

In [31]:
feature_cols = np.concatenate((np.array(['adult', 'video']), genre_cols,countries_cols,collection_cols,keyword_cols,companies_cols,lang_cols,credit_cols))
feature_cols
#metadata[feature_cols] = metadata[feature_cols].astype('int8')

array(['adult', 'video', 'genre:', ..., 'cast:woody harrelson',
       'cast:xander berkeley', 'cast:yuen biao'], dtype=object)

In [32]:
del genre_data,countries_data,collection_data,keyword_data,companies_data,lang_data,credit_data
del genre_cols,countries_cols,collection_cols,keyword_cols,companies_cols,lang_cols,credit_cols

In [33]:
feature_cols.shape

(3672,)

In [34]:
metadata

,title,id,adult,video,genre:,genre:action,genre:adventure,genre:animation,genre:comedy,genre:crime,...,cast:william hurt,cast:william powell,cast:william sadler,cast:william schallert,cast:william shatner,cast:winona ryder,cast:woody allen,cast:woody harrelson,cast:xander berkeley,cast:yuen biao
0,Toy Story,862,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Jumanji,8844,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,15602,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,31357,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,11862,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46159,Subdue,439050,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46160,Century of Birthing,111109,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46161,Betrayal,67758,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46162,Satan Triumphant,227506,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
def split_dataframe(df, holdout_fraction=0.1):
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

train, test = split_dataframe(metadata)

In [36]:
allIds = metadata['id']

number_of_batches = 4
batches = np.array_split(train, number_of_batches)
mf.log_param('number of batches', number_of_batches)
del metadata
del train

## Algorithm


In [37]:
batches[0]

,title,id,adult,video,genre:,genre:action,genre:adventure,genre:animation,genre:comedy,genre:crime,...,cast:william hurt,cast:william powell,cast:william sadler,cast:william schallert,cast:william shatner,cast:winona ryder,cast:woody allen,cast:woody harrelson,cast:xander berkeley,cast:yuen biao
0,Toy Story,862,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Grumpier Old Men,15602,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Waiting to Exhale,31357,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Father of the Bride Part II,11862,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,Heat,949,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11542,The Bothersome Man,13318,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11543,Don't Drink the Water,10462,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
11544,The Good German,182,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
11546,Letters from Iwo Jima,1251,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
from sklearn.metrics.pairwise import cosine_similarity

`content_based_recommmeder` returns a list of movie ids based on it's input. the input should be a dataframe which has `movieId`, `rating` columns(like `ratings_small.csv` but without `userId`)

In [39]:
number_of_batches =1
def content_based_recommender_movie(movieId):
    print("movie title is:", metadata[metadata['id']==movieId])
    sim_mat= cosine_similarity(metadata[feature_cols])
    return sim_mat

#content_based_recommender_movie(272)

In [40]:
batches[1].describe()

,id,adult,video,genre:,genre:action,genre:adventure,genre:animation,genre:comedy,genre:crime,genre:documentary,...,cast:william hurt,cast:william powell,cast:william sadler,cast:william schallert,cast:william shatner,cast:winona ryder,cast:woody allen,cast:woody harrelson,cast:xander berkeley,cast:yuen biao
count,10387.000000,10387.000000,10387.0,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,...,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000,10387.000000
mean,70745.557909,0.000096,0.0,0.039569,0.151054,0.071628,0.038125,0.265813,0.094541,0.111678,...,0.001637,0.000674,0.001348,0.001252,0.000674,0.001059,0.000578,0.002214,0.000770,0.001155
std,63999.684940,0.009812,0.0,0.194953,0.358119,0.257883,0.191506,0.441786,0.292594,0.314985,...,0.040424,0.025952,0.036690,0.037983,0.025952,0.032527,0.024028,0.047007,0.027743,0.036697
min,3.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25769.500000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,50675.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,94217.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,469172.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [41]:
from sklearn.metrics.pairwise import euclidean_distances as dist
def content_based_recommender(user, df, k=10, movieIds=allIds):
    user_movies = pd.merge(user,df,how='inner',left_on='movieId',right_on='id')
    user_movies[feature_cols] = user_movies[feature_cols].multiply(user_movies['rating'], axis="index")
    mean_user_movies = user_movies[feature_cols].mean(axis=0)
    sim_mat = cosine_similarity(df[feature_cols][df.id.isin(movieIds)], mean_user_movies[feature_cols].values.reshape(1,-1))
    temp_data = {'id':df['id'][df.id.isin(movieIds)], 'title':df['title'][df.id.isin(movieIds)], 'sim':sim_mat.flatten()}
    return pd.DataFrame(temp_data)

def content_based_all_batches(user, k=10, movieIds=allIds):
    ans = content_based_recommender(user, batches[0], k, movieIds)
    for i in range(1,number_of_batches):
        ans.append(content_based_recommender(user, batches[i], k, movieIds))
    return ans.sort_values(by='sim', ascending=False)
    

content_based_k = 10
mf.log_param('content based k', content_based_k)
#xx = content_based_recommender(rating[rating['userId'] == 1], batches[1], content_based_k)
xx = content_based_all_batches(rating[rating['userId'] == 1], content_based_k)
xx.shape

(10387, 3)

# Collaborative Filtering

### import libraries

In [42]:
import numpy as np
import pandas as pd
from sklearn.utils.extmath import randomized_svd

### explore datasets

In [43]:
rating = pd.read_csv('/content/IMDB/ratings_small.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [44]:
rating.shape

(100004, 4)

In [45]:
links_small = pd.read_csv('/content/IMDB/links_small.csv')
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [46]:
credits = pd.read_csv('/content/IMDB/credits.csv')
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [47]:
movie = pd.read_csv('/content/IMDB/movies_metadata.csv')
movie.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [48]:
movie = movie.rename(columns={'id': 'movieId'})

In [49]:
movie.shape

(45466, 24)

In [50]:
movie.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### data preprocessing

There are three rows entered by mistake, so we remove that row.

In [51]:
movie = movie[(movie['movieId']!='1997-08-20') & (movie['movieId']!='2012-09-29') & (movie['movieId']!='2014-01-01')]

In [52]:
def find_names(x):
    if x == '':
        return ''
    genre_arr = eval(str(x))
    return ','.join(i['name'] for i in eval(str(x)))
    
movie['genres'] = movie['genres'].fillna('')

<ipython-input-52-09947835ec44>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie['genres'] = movie['genres'].fillna('')


In [53]:
movie['genres']=movie['genres'].apply(find_names)

In [54]:
movie.movieId = movie.movieId.astype("uint64")

only keep rating for movies with metadata in movie dataset

In [55]:
new_rating = pd.merge(rating, movie,  how='inner', on=["movieId"])

In [56]:
new_rating = new_rating[["userId", "movieId", "rating"]]

In [57]:
movie.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"Animation,Comedy,Family",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"Adventure,Fantasy,Family",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"Romance,Comedy",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"Comedy,Drama,Romance",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [58]:
new_rating.head()

,userId,movieId,rating
0,1,1371,2.5
1,4,1371,4.0
2,7,1371,3.0
3,19,1371,4.0
4,21,1371,3.0


In [ ]:
train, test = split_dataframe(new_rating)

### matrix factorization

In [59]:
inter_mat_df = rating.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
inter_mat_df

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
inter_mat = inter_mat_df.to_numpy()

In [61]:
ratings_mean = np.mean(inter_mat, axis = 1)
inter_mat_normal = inter_mat - ratings_mean.reshape(-1, 1)

In [62]:
inter_mat_normal

array([[-0.00562541, -0.00562541, -0.00562541, ..., -0.00562541,
        -0.00562541, -0.00562541],
       [-0.02923009, -0.02923009, -0.02923009, ..., -0.02923009,
        -0.02923009, -0.02923009],
       [-0.02007501, -0.02007501, -0.02007501, ..., -0.02007501,
        -0.02007501, -0.02007501],
       ...,
       [-0.01367748, -0.01367748, -0.01367748, ..., -0.01367748,
        -0.01367748, -0.01367748],
       [ 3.98698434, -0.01301566, -0.01301566, ..., -0.01301566,
        -0.01301566, -0.01301566],
       [ 4.95030885, -0.04969115, -0.04969115, ..., -0.04969115,
        -0.04969115, -0.04969115]])

We use singular value decomposition for matrix factorization

In [63]:
svd_U, svd_sigma, svd_V = randomized_svd(inter_mat_normal, 
                              n_components=15,
                              n_iter=5,
                              random_state=47)

This function gives the diagonal form

In [64]:
svd_sigma = np.diag(svd_sigma)

Making predictions

In [65]:
rating_weights = np.dot(np.dot(svd_U, svd_sigma), svd_V) + ratings_mean.reshape(-1, 1)

In [66]:
weights_df = pd.DataFrame(rating_weights, columns = inter_mat_df.columns)

In [67]:
weights_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.081106,0.024332,-0.016835,-0.006440,-0.028982,0.031704,-0.000823,-0.004803,-0.005659,0.039295,...,-0.004629,-0.004407,0.011158,-0.004547,-0.004340,-0.005154,0.019218,-0.005677,-0.005391,-0.004297
1,1.010433,1.676149,0.180399,0.133225,0.332127,0.659165,0.161904,0.061847,0.088149,2.271430,...,-0.000064,-0.001081,0.006137,-0.003853,-0.003784,-0.004673,0.011201,-0.002404,-0.003039,0.003864
2,1.023170,0.389789,-0.031488,0.031783,0.000714,0.135239,-0.080027,0.030734,-0.026625,0.397392,...,0.003256,0.007455,-0.012356,-0.000387,-0.006532,-0.000590,-0.024486,0.015854,0.014274,-0.005391
3,1.980784,1.192729,0.128675,0.106373,-0.234511,-0.550348,-0.085587,-0.143159,-0.015308,1.372206,...,0.032565,-0.021875,0.084978,0.008436,0.018263,-0.022110,0.133075,-0.016431,-0.014345,0.076347
4,1.216316,0.926650,0.122319,0.063811,0.628848,-0.137074,0.356099,0.082625,-0.068154,0.567814,...,-0.028169,-0.028221,0.000680,-0.024315,-0.023941,-0.061806,0.013551,-0.023701,-0.024246,-0.031276


making recommendations

In [68]:
def recommend_top_k(preds_df, ratings_df, movie, userId, k=10):
    user_row = userId-1 
    sorted_user_predictions = preds_df.iloc[user_row].sort_values(ascending=False) 
    user_data = ratings_df[ratings_df.userId == (userId)]
    user_rated = user_data.merge(movie, how = 'left', left_on = 'movieId', right_on = 'movieId'). \
                  sort_values(['rating'], ascending=False)
    user_preds = movie.merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               on = 'movieId').rename(columns = {user_row: 'prediction'}). \
               sort_values('prediction', ascending = False). \
               iloc[:k, :]
    return user_rated, user_preds

In [69]:
collaborative_k = 100
user_rated, user_preds = recommend_top_k(weights_df, new_rating, movie, 220, collaborative_k)
mf.log_param('collaborative k', collaborative_k)

100

In [70]:
user_preds.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,prediction
6388,False,"{'id': 528, 'name': 'The Terminator Collection...",200000000,"Action,Thriller,Science Fiction",NaN,296,tt0181852,en,Terminator 3: Rise of the Machines,It's been 10 years since John Connor saved Ear...,...,435000000.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Machines Will Rise.,Terminator 3: Rise of the Machines,False,5.9,2177.0,4.792743
3382,False,NaN,0,"Drama,Science Fiction,Adventure,Mystery",NaN,593,tt0069293,ru,Солярис,Ground control has been receiving strange tran...,...,0.0,167.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,Solaris,False,7.7,364.0,4.742942
5325,False,"{'id': 86055, 'name': 'Men In Black Collection...",140000000,"Action,Adventure,Comedy,Science Fiction",http://www.sonypictures.com/homevideo/meninbla...,608,tt0120912,en,Men in Black II,"Kay and Jay reunite to provide our best, last ...",...,441818803.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Same Planet. New Scum.,Men in Black II,False,6.1,3188.0,4.647800
4020,False,NaN,8000000,"Drama,Thriller",NaN,318,tt0120753,en,The Million Dollar Hotel,The Million Dollar Hotel starts with a jump fr...,...,0.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Million Dollar Hotel,False,5.9,76.0,4.469385
286,False,"{'id': 300546, 'name': 'Once were Warriors Col...",0,Drama,NaN,527,tt0110729,en,Once Were Warriors,A drama about a Maori family lving in Auckland...,...,2201126.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,"A family in crisis, a life in chaos... Nothing...",Once Were Warriors,False,7.6,106.0,4.236960


In [71]:
user_rated.head()

,userId,movieId,rating,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,220,2294,5.0,False,NaN,22000000,Comedy,NaN,tt0261392,en,...,2001-08-22,33788161.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Hollywood had it coming,Jay and Silent Bob Strike Back,False,6.4,491.0
46,220,1247,5.0,False,NaN,85000000,"Drama,Thriller,History",http://www.thegoodshepherdmovie.com/,tt0343737,en,...,2006-12-11,59908565.0,167.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The untold story of the most powerful covert a...,The Good Shepherd,False,6.3,342.0
25,220,2762,5.0,False,NaN,0,"Drama,Crime",NaN,tt0029811,en,...,1937-11-01,0.0,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A Brilliant Melodrama,Young and Innocent,False,6.8,42.0
27,220,260,5.0,False,NaN,0,"Action,Thriller,Mystery",NaN,tt0026029,en,...,1935-06-01,0.0,86.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Handcuffed to the girl who double-crossed him,The 39 Steps,False,7.4,217.0
59,220,2324,5.0,False,NaN,3250000,Drama,http://www.localcolormovie.com/,tt0472126,en,...,2006-09-19,32788.0,107.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Local Color,False,6.1,8.0


In [72]:
user_rated[["title", "genres"]].head(10)

,title,genres
0,Jay and Silent Bob Strike Back,Comedy
46,The Good Shepherd,"Drama,Thriller,History"
25,Young and Innocent,"Drama,Crime"
27,The 39 Steps,"Action,Thriller,Mystery"
59,Local Color,Drama
31,The Big Sleep,"Crime,Drama,Mystery,Thriller"
33,The Talented Mr. Ripley,"Thriller,Crime,Drama"
42,The Big Parade,"Drama,Romance,War"
73,Dancer in the Dark,"Drama,Crime,Music"
110,Birdman of Alcatraz,Drama


In [73]:
user_preds[["title", "genres"]].head(10)

,title,genres
6388,Terminator 3: Rise of the Machines,"Action,Thriller,Science Fiction"
3382,Solaris,"Drama,Science Fiction,Adventure,Mystery"
5325,Men in Black II,"Action,Adventure,Comedy,Science Fiction"
4020,The Million Dollar Hotel,"Drama,Thriller"
286,Once Were Warriors,Drama
2100,Young and Innocent,"Drama,Crime"
534,Sleepless in Seattle,"Comedy,Drama,Romance"
2137,Say Anything...,"Comedy,Drama,Romance"
11922,License to Wed,Comedy
33911,The Tunnel,Science Fiction


# Ensemble Model

In [74]:
def ensemble(userId, k=10):
    user_rated, user_preds = recommend_top_k(weights_df, new_rating, movie, userId, k*k)
    content_based_result = content_based_all_batches(rating[rating['userId'] == userId], k=k, movieIds=user_preds['movieId'])
    return content_based_result[['id','title']]

In [75]:
ensemble_k=10
mf.log_param('ensemble k', ensemble_k)
ensemble(220, ensemble_k)

,id,title
2663,912,The Thomas Crown Affair
1415,1968,Fools Rush In
2077,1580,Rope
2110,2762,Young and Innocent
533,858,Sleepless in Seattle
...,...,...
255,11,Star Wars
1315,377,A Nightmare on Elm Street
1344,364,Batman Returns
5524,1682,Mothra vs. Godzilla


# Evaluation

In [76]:
df_res = user_preds[["movieId", "prediction"]]. \
          merge(user_rated[["movieId", "rating"]], how = 'outer', on = 'movieId')

In [77]:
df_res.sort_values(by='prediction',ascending=False,inplace=True)
df_res

,movieId,prediction,rating
0,296.0,4.792743,5.0
1,593.0,4.742942,4.0
2,608.0,4.647800,5.0
3,318.0,4.469385,NaN
4,527.0,4.236960,5.0
...,...,...,...
174,2269.0,NaN,1.0
175,586.0,NaN,1.0
176,344.0,NaN,1.0
177,2054.0,NaN,1.0


In [78]:
threshold = 2
df_res['prediction'] = df_res['prediction'] >= threshold
df_res['rating'] = df_res['rating'] >= threshold
df_res

,movieId,prediction,rating
0,296.0,True,True
1,593.0,True,True
2,608.0,True,True
3,318.0,True,False
4,527.0,True,True
...,...,...,...
174,2269.0,False,False
175,586.0,False,False
176,344.0,False,False
177,2054.0,False,False


In [79]:
def precision_at_k(df, k=10, y_test: str='rating', y_pred='prediction'):  
    dfK = df.head(k)
    sum_df = dfK[y_pred].sum()
    true_pred = dfK[dfK[y_pred] & dfK[y_test]].shape[0]
    if sum_df > 0:
        return true_pred/sum_df
    else:
        return None

def recall_at_k(df, k=10, y_test='rating', y_pred='prediction'):
    dfK = df.head(k)
    sum_df = df[y_test].sum()
    true_pred = dfK[dfK[y_pred] & dfK[y_test]].shape[0]
    if sum_df > 0:
        return true_pred/sum_df
    else:
        return None

In [80]:
prec_at_k = precision_at_k(df_res, 100, y_test='rating', y_pred='prediction')
rec_at_k = recall_at_k(df_res, 100, y_test='rating', y_pred='prediction')

In [81]:
print("precision@k: ", prec_at_k)
print("recall@k: ", rec_at_k)
mf.log_metric('recall', rec_at_k)
mf.log_metric('precision', prec_at_k)

precision@k:  0.7941176470588235
recall@k:  0.21774193548387097


# MLOps

In [82]:
def updata_batch(new_batch):
    number_of_batches = number_of_batches+1
    batches = batches.append(new_batch)
    mf.log_param('number of batches', number_of_batches)